# Notebook info

# Package setup

## System-level setup

In [0]:
#@title
# Useful for Colab
!pip install geopandas
!pip install rtree
!sudo apt-get update && apt-get install -y libspatialindex-dev
!pip install spatiallibindex

In [0]:
#@title
# This setup is useful for Colab and other environments with pesky setup for Geopandas
!apt-get install -qq curl g++ make
!curl -L http://download.osgeo.org/libspatialindex/spatialindex-src-1.8.5.tar.gz | tar xz
import os
os.chdir('spatialindex-src-1.8.5')
!make
!make install
!pip install rtree
!ldconfig
from rtree import index
from rtree.index import Rtree

## Import libraries

In [0]:
# Importing analysis libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt

import geopandas as gpd
import shapely.geometry
from shapely.geometry import Point

In [0]:
import os 
import pandas as pd
## We use pip since Colab doenst have conda
import geopandas as gpd
import fiona
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime

# high-performance plotting
# import bokeh
# from bokeh.io import output_notebook, show, output_file
# from bokeh.plotting import figure
# from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
# from bokeh.palettes import brewer
# import json


In [13]:
#@title
# PATH MANAGEMENT
# OS-independent setup
import os
from pathlib import Path
# https://docs.python.org/3/library/pathlib.html
CWD = Path.cwd()
HOME = Path.home()
DATA_IN = CWD / 'data_in'
DATA_OUT = CWD / 'data_out'
VIZ = CWD / "viz"

print(f"""PATH SETUP: 
---
cwd is {CWD}

Refer to these paths for IO operations:
{HOME}
{DATA_IN}
{DATA_OUT}
{VIZ}
""")

PATH SETUP: 
---
cwd is /content

Refer to these paths for IO operations:
/root
/content/data_in
/content/data_out
/content/viz



## Jupyter notebook options and viz options

In [0]:
# Options for python notebook
pd.options.display.max_rows = 1000
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [0]:
# Theming for plots + high DPI figures
%config InlineBackend.figure_format = 'retina'


# Analysis

# Load and verify data

# Visualization

In [0]:
df['geometry'] = list(zip(df.longitude, df.latitude))

In [0]:
%%time
%store
df['geometry'] = df['geometry'].apply(Point)
gdf = gpd.GeoDataFrame(df, geometry='geometry')
gdf.sample(4)
print(gdf.crs)

In [0]:
CRS_WGS = {'init': 'epsg:4326', 'no_defs': True}
CRS_LA = {'init': 'epsg:2229'}

In [0]:
gdf.crs = CRS_WGS

In [0]:
gdf.crs

In [0]:
airbnb_crs_la = gdf.to_crs(crs=CRS_LA)

In [0]:
gdf.plot(figsize=(10, 50), color='teal', markersize=5)


In [0]:
airbnb_crs_la.plot(figsize=(10, 50), color='pink', markersize=5)

In [0]:
#neighborhoods base file

neighborhoods=gpd.read_file("/content/drive/My Drive/Colab Notebooks/IFX_Final_HomelessnessLA/data_in/LA-Neighborhoods.geojson")

In [0]:
neighborhoods.crs

In [0]:
neighborhoods_crs_la =  neighborhoods.to_crs(crs=CRS_LA)

In [0]:
neighborhoods_crs_la

In [0]:
neighborhoods_crs_la.crs

In [0]:
neighborhoods_crs_la.plot()

In [0]:
print(f'neighborhood:{neighborhoods_crs_la.crs}, airbnb:{airbnb_crs_la.crs}')

In [0]:
#43560 feet = 1 acre
neighborhoods_crs_la['neighborhood_acres'] = neighborhoods_crs_la['geometry'].area / 43560

In [0]:
#might have to restart kernel
%%time
%store
airbnb_la = gpd.sjoin(
    airbnb_crs_la,
    neighborhoods_crs_la,
    how='inner')

In [0]:
airbnb_la.head()

In [0]:
%%time
%store
## Count points in neighborhood
point_counts = airbnb_la.groupby(['neighbourhood_left'], as_index=False).count()


In [0]:
point_counts.head()

In [0]:
point_counts['points_in_neighborhood'] = point_counts['index_right']


In [0]:
# ## Join points to the original 
df2 = airbnb_la.merge(
    point_counts[['neighbourhood_left', 'points_in_neighborhood', 'neighborhood_acres']],
    on='neighbourhood_left'
    ).drop(['index_right', 'neighbourhood_group', 'neighbourhood_right'], axis='columns')


In [0]:
df2.head()

In [0]:
df2['units_per_area'] = df2['points_in_neighborhood'] / df2['neighborhood_acres_x']

In [0]:
df2.sort_values('units_per_area',ascending=False).head()
summary_unit_density = df2.groupby(['neighbourhood_left'], as_index=False).mean().sort_values(['units_per_area'], ascending=False)


In [0]:
summary_unit_density.head(10)

In [0]:
summary_unit_density.to_csv('/content/drive/My Drive/Colab Notebooks/IFX_Final_HomelessnessLA/data_out/airbnb_density_la.csv')

# Export/saving